<a href="https://colab.research.google.com/github/wonmadeit/LEEGAWON/blob/main/6_Specifying_batch_size_while_training_a_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Batch size : loss를 계산하고, weight를 1회 update(1epoch)할 때 사용할 데이터의 양
  - 모델 & 학습 설계한 사람이 설정할 수 있는 hyperparameter 중 하나

In [1]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn

In [2]:
x = [[1,2],[3,4],[5,6],[7,8]]
y = [[3],[7],[11],[15]]

In [3]:
X = torch.tensor(x).float()
Y = torch.tensor(y).float()

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
X = X.to(device)
Y = Y.to(device)

- Dataset을 상속받은 MyDataset에 data에 대한 정보를 저장
  - `__init__`에 input & output 쌍을 tensor type으로 변환하여 저장
  - `__len__`에 dataset의 length, `__getitem__`에 특정 row를 추출하기 위한 함수를 정의
- DataLoader를 사용해 data point들을 batch size 만큼 batch로 묶음

In [6]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x = torch.tensor(x).float()
        self.y = torch.tensor(y).float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, ix):
        return self.x[ix], self.y[ix]     #해당 인덱스 아이템 리턴
ds = MyDataset(X, Y) # dataset instance 생성 , 이 인스턴스를 ds가 참조

<ipython-input-6-f864dabbc481>:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x = torch.tensor(x).float()
<ipython-input-6-f864dabbc481>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y = torch.tensor(y).float()


In [11]:
dl = DataLoader(ds, batch_size=2, shuffle=True) # batch를 ds의 순서대로 구성할지, 섞어서 batch를 구성할지 shuffle argument를 통해 정의
                                                # 학습데이터는 suffle = true 지정이 일반적, validation, test데이터는 shuffle x
                                                # 데이터 순서 강아지 1~100, 고양이 101~200 -> 학습 bad -> suffle!
                                                #

In [12]:
class MyNeuralNet(nn.Module):   #모델 정의 클래스
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer = nn.Linear(2,8)
        self.hidden_layer_activation = nn.ReLU()
        self.hidden_to_output_layer = nn.Linear(8,1)
    def forward(self, x):
        x = self.input_to_hidden_layer(x)
        x = self.hidden_layer_activation(x)
        x = self.hidden_to_output_layer(x)
        return x

In [13]:
mynet = MyNeuralNet().to(device)
loss_func = nn.MSELoss()
from torch.optim import SGD
opt = SGD(mynet.parameters(), lr = 0.001)

In [28]:
import time
loss_history = []
start = time.time()
for _ in range(50): # epoch
    for data in dl: # step (batch)                        # mini-batch 하나씩 받아옴
        x, y = data                                       # 샘플 두개 포함(x,y)
        opt.zero_grad()                                   #
        loss_value = loss_func(mynet(x),y)                #
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end = time.time()
print(end - start)

0.12329459190368652


- Validation 용 input 정의 후 추정값 확인

In [24]:
val_x = [[10,11]]

In [25]:
val_x = torch.tensor(val_x).float().to(device)

In [29]:
mynet(val_x)

tensor([[20.7914]], device='cuda:0', grad_fn=<AddmmBackward0>)